In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
R.<x> = PolynomialRing(QQ)
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return xjNew(n,m)/x
def H4(n,m):
    jay=j(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-2^6*m^3)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
def H6(n,m): #different than in draft 4mar21 and Mathematica.
    def base6(n,m):
        jay=j(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(j(n+1,m)-2^6*m^3))
        return L((-1)^m*(num/den)).O(n+1)
    return (S(base6(n,m)^(1/(m-2))).polynomial())
def deltaStar6(n,m):
    jay=j(n,m)
    djay=x*derivative(jay,x)
    numerator=L(djay^(2*m))
    denominator=L(jay^(2*m-2)*(jay-2^6*m^3)^m)
    base=L(numerator/denominator)
    xp=1/(m-2)
    power=S(base^xp)
    return D(power).truncate(n+1)
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def deltaDiamond(n,m):
    return D(S(expand((H4(n+2,m)^3)/j(n+2,m)))).truncate(n+1)
def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)
def deltaDagger(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return (ddstep1(n,m)-ddstep2(n,m))/1728
def last(list):return list[-1]
def quotientFOvrG(f,g) : 
    return (f.maxima_methods().divide(g))[0]
def remainderFOvrG(f,g) : 
    return (f.maxima_methods().divide(g))[1]
def factor(polynomial):
    fp=polynomial.factor()
    return fp
def numberOfFactors(polynomial):
    fp=polynomial.factor()
    return len(fp)
def factorK(polynomial,k):
    fp=factor(h)
    return fp[k-1]
def factorList(polynomial):
    answer=[]
    nf=numberOfFactors(polynomial)
    for k in [1..nf]:
        answer=answer+[factorK(polynomial,k)]
    return answer
def polynomialExponentList(polynomial):
    pel=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        pel=pel+[exponent]
    return pel
def polynomialFactorBaseList(polynomial):
    fbl=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        fbl=fbl+[base]
    return fbl
def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))
def primeFactorsIn(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    return flatten(primelist)
def den(a):
    if a in ZZ:return 1
    else:return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def substituteNumberForXInExpression(number,x,Xexpression):
    var('x')
    return sage_eval(str(Xexpression),locals={'x':number})
def differenceTable(list):
    lnth=len(list)
    dt=[]
    for k in [1..lnth-1]:
        diff=list[k]-list[k-1]
        dt=dt+[diff]
    return dt

In [3]:
for m in [3..12]:
    print "-----------------------------------------------------------------------"
    print deltaStar6(5,m)

-----------------------------------------------------------------------
4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x
-----------------------------------------------------------------------
-13762560*x^5 + 262144*x^4 + 3072*x^3 - 128*x^2 + x
-----------------------------------------------------------------------
-12041250*x^5 + 7688000/3*x^4 + 27900*x^3 - 360*x^2 + x
-----------------------------------------------------------------------
1610612736*x^5 + 8388608*x^4 + 147456*x^3 - 768*x^2 + x
-----------------------------------------------------------------------
22718141950*x^5 - 11524800*x^4 + 543900*x^3 - 1400*x^2 + x
-----------------------------------------------------------------------
195066593280*x^5 - 740294656/3*x^4 + 1585152*x^3 - 2304*x^2 + x
-----------------------------------------------------------------------
1239560731710*x^5 - 1414276416*x^4 + 3921372*x^3 - 3528*x^2 + x
-----------------------------------------------------------------------
18790481920000/3*x^5 - 1677721

In [5]:
poly=deltaStar6(100,3)
data=[]
ls=poly.list()
for n in [1..len(ls)-1]:
    data=data+[polynomialCoefficient(n,poly)-nieburTau(n)]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
import time
import pickle
wfile = open('/Users/barrybrent/21mar21no6.txt','w')
data=[]
for m in [3..203]:
    start=time.time()
    ds=deltaStar6(50,m)
    data=data+[[m,ds]]
    finish=time.time()
    print([m,ds.degree(),finish-start])
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3, 50, 0.24636006355285645]
[4, 50, 0.1733407974243164]
[5, 50, 0.3544039726257324]
[6, 50, 0.26319289207458496]
[7, 50, 0.49260997772216797]
[8, 50, 0.4555790424346924]
[9, 50, 0.6408579349517822]
[10, 50, 0.6091749668121338]
[11, 50, 0.8461639881134033]
[12, 50, 0.8435859680175781]
[13, 50, 1.192126989364624]
[14, 50, 1.0814261436462402]
[15, 50, 1.456017017364502]
[16, 50, 1.4126200675964355]
[17, 50, 2.0155060291290283]
[18, 50, 1.7939469814300537]
[19, 50, 2.3202600479125977]
[20, 50, 2.2177629470825195]
[21, 50, 2.4740490913391113]
[22, 50, 2.620087146759033]
[23, 50, 3.351773977279663]
[24, 50, 3.2391538619995117]
[25, 50, 4.1443140506744385]
[26, 50, 4.432799816131592]
[27, 50, 4.875384092330933]
[28, 50, 4.837494850158691]
[29, 50, 5.915580987930298]
[30, 50, 4.9032299518585205]
[31, 50, 6.850108861923218]
[32, 50, 6.297735929489136]
[33, 50, 7.945516109466553]
[34, 50, 7.8884570598602295]
[35, 50, 8.762682914733887]
[36, 50, 8.690347909927368]
[37, 50, 11.059343814849854]
[3

In [5]:
import pickle
rfile = open('/Users/barrybrent/21mar21no6.txt','r') # star series
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
s[3]

[6,
 332091432412187385907773081824444370005652621165975955838691183978103348323624823693513467682367730270339072*x^50 - 2128737393393616789371689047537770758587241442277957217326508391147273936564794218525437752511333088100352*x^49 - 11181225705090756957392231385025280419685402130726101975008539339967679731907209719518599360941851148288*x^48 - 159532599738860932233919381190391508335446795365773682081804643605025946879104481400114992349118988288*x^47 + 336417958157543337977215918721704458049925962271172811266659053231999606810450886628616528692510720*x^46 + 253439868031631420909174938769141193285379491666341124559146050146874703791355299636513567932416*x^45 - 9191106325221201221243226946103114262664225392220704362870265707486968424532175218299488960512*x^44 + 306955858683902872124576409672188171643443143570535768767733413962088332108087799913010692096*x^43 + 537109095322569573527812827998566507308492642861135534835701095044612256773536045575700480*x^42 - 1334613528959075871721670329987

In [6]:
# interpolating polynomials
import pickle
rfile = open('/Users/barrybrent/21mar21no6.txt','r') # star series
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/22mar21no2.txt','w') # star polynomials
for qpower in [1..50]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=s[k][1] 
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower,rl]] 
    finish = time.time()
    print([qpower,rl.degree(),finish-start])
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

[1, 0, 0.7311859130859375]
[2, 3, 0.27023983001708984]
[3, 6, 0.27632594108581543]
[4, 9, 0.31366395950317383]
[5, 12, 0.27547407150268555]
[6, 15, 0.2704010009765625]
[7, 18, 0.2724180221557617]
[8, 21, 0.31208205223083496]
[9, 24, 0.27060699462890625]
[10, 27, 0.2686750888824463]
[11, 30, 0.2706921100616455]
[12, 33, 0.2754981517791748]
[13, 36, 0.32024192810058594]
[14, 39, 0.27466702461242676]
[15, 42, 0.27213621139526367]
[16, 45, 0.27494311332702637]
[17, 48, 0.3207869529724121]
[18, 51, 0.27934813499450684]
[19, 54, 0.28264498710632324]
[20, 57, 0.27942419052124023]
[21, 60, 0.32290196418762207]
[22, 63, 0.28374195098876953]
[23, 66, 0.2867250442504883]
[24, 69, 0.2891380786895752]
[25, 72, 0.29083704948425293]
[26, 75, 0.3346531391143799]
[27, 78, 0.293834924697876]
[28, 81, 0.293881893157959]
[29, 84, 0.29825901985168457]
[30, 87, 0.3430299758911133]
[31, 90, 0.298382043838501]
[32, 93, 0.30766892433166504]
[33, 96, 0.30045199394226074]
[34, 99, 0.35308003425598145]
[35, 102, 

In [7]:
import pickle
rfile = open('/Users/barrybrent/22mar21no2.txt','r') # star polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
s[3]

[4,
 -64*x^9 + 3584/3*x^8 - 242432/27*x^7 + 323584/9*x^6 - 2204672/27*x^5 + 892928/9*x^4 - 1347584/27*x^3]

In [3]:
import pickle
rfile = open('/Users/barrybrent/22mar21no2.txt','r') # star polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
numerators=[]
denominators=[]
for k in [1..5]:
    n=s[k][0]
    poly=D(s[k][1])
    f = lastfactor(poly)
    print(f)
    value=substituteNumberForXInExpression(3,x,f)
    data=data+[value]
    numerators=numerators+[num(value)]
    denominators=denominators+[den(value)]
print(data)
print(".................................................................................................")
print(numerators)
print(".................................................................................................")
print(denominators)

x^2 - 4*x + 4
x^2 - 44/7*x + 76/7
x^4 - 44/3*x^3 + 2096/27*x^2 - 5200/27*x + 5264/27
x^6 - 4220/189*x^5 + 364300/1701*x^4 - 629344/567*x^3 + 5611696/1701*x^2 - 3141440/567*x + 7226176/1701
x^8 - 3128/105*x^7 + 1078208/2625*x^6 - 25985824/7875*x^5 + 14834912/875*x^4 - 450196352/7875*x^3 + 980173312/7875*x^2 - 432644608/2625*x + 830809856/7875
[1, 1, 23/27, 115/243, 1/9]
.................................................................................................
[1, 1, 23, 115, 1]
.................................................................................................
[1, 1, 27, 243, 9]


In [2]:
import pickle
rfile = open('/Users/barrybrent/22mar21no2.txt','r') # star polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
numerators=[]
denominators=[]
for k in [1..30]:
    n=s[k][0]
    poly=D(s[k][1])
    f = lastfactor(poly)
    value=substituteNumberForXInExpression(3,x,f)
    data=data+[value]
    numerators=numerators+[num(value)]
    denominators=denominators+[den(value)]
print(data)
print(".................................................................................................")
print(numerators)
print(".................................................................................................")
print(denominators)

[1, 1, 23/27, 115/243, 1/9, -2093/31347, -55/729, -1403/61317, 115/19683, 44551/6554439, 23/19683, -41267/83436237, -2093/22851963, 115/1594323, -241/14348907, -54809/1678822119, -1403/402300837, 10879/2711943423, 2645/3486784401, -2093/16659081027, 44551/387033068511, 776803/15910197221763, -55/3486784401, -25499225/4448547629112231, -41267/44341437227517, -359/254535261273, -48139/327900691854441, 4280221/18598832265883293, 115/7625597484987, -33701/3911931509798331]
.................................................................................................
[1, 1, 23, 115, 1, -2093, -55, -1403, 115, 44551, 23, -41267, -2093, 115, -241, -54809, -1403, 10879, 2645, -2093, 44551, 776803, -55, -25499225, -41267, -359, -48139, 4280221, 115, -33701]
.................................................................................................
[1, 1, 27, 243, 9, 31347, 729, 61317, 19683, 6554439, 19683, 83436237, 22851963, 1594323, 14348907, 1678822119, 402300837, 2711943423, 34867

In [8]:
import pickle
rfile = open('/Users/barrybrent/22mar21no2.txt','r') # star polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
numerators=[]
denominators=[]
for k in [1..30]:
    n=s[k][0]
    poly=D(s[k][1])
    f = lastfactor(poly)
    value=substituteNumberForXInExpression(n,x,f)
    data=data+[value]
    numerators=numerators+[num(value)]
    denominators=denominators+[den(value)]
print(data)
print(".................................................................................................")
print(numerators)
print(".................................................................................................")
print(denominators)

[0, 1, -16, -3211/189, 4194304/63, -1390116161/215, -18706141184/15, 1492541185158832223/2146095, -9889764044217253888/127575, -40530335334176770285589/524475, 392573681964595409037819904/4822335, -2038481837245643166522867119/40425, -547977773602647201834219033591808/20114325, 36115557338191016972048276573438795686280693/342124776118125, -26054871227588924932891715144192622592/375375, -813188283199234991978718500936229088005177649/8300667375, 6527280144468707674481224643995868273540898758852608/27105018892875, -8820693867256675267366666964843738041676516541449971/27353891565, -146545386619294828518191310777869174887688714956115542016/1055319890625, 18700790490414304918076910128610915703743246636197374690953634555/6137835783482403, -1500254146015107840777892915462179231140948170535643776300930301952/326431548568125, -96286073468776614143083631450606316742437712830501443959327242350990269/5217300741423771, 130276057580206405504938495857716805342627943861099428410641637319327268995072/18

In [45]:
nbr=6
list=nbr.digits(2)
print list
list.reverse()
print list
a=QQ(3/2)
list=a.n(digits=2)
print list


[0, 1, 1]
[1, 1, 0]
1.5


In [46]:
pi.n(digits=10)

3.141592654

In [38]:
import pickle
rfile = open('/Users/barrybrent/22mar21no2.txt','r') # star polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..30]:
    n=s[k][0]
    poly=D(s[k][1])
    f = lastfactor(poly)
    value=abs(num(substituteNumberForXInExpression(3,x,f)))
    vd3=value.digits(3)
    vd3.reverse() 
    print([n,vd3])

[2, [1]]
[3, [1]]
[4, [2, 1, 2]]
[5, [1, 1, 0, 2, 1]]
[6, [1]]
[7, [2, 2, 1, 2, 1, 1, 2]]
[8, [2, 0, 0, 1]]
[9, [1, 2, 2, 0, 2, 2, 2]]
[10, [1, 1, 0, 2, 1]]
[11, [2, 0, 2, 1, 0, 1, 0, 0, 0, 1]]
[12, [2, 1, 2]]
[13, [2, 0, 0, 2, 1, 2, 1, 1, 0, 2]]
[14, [2, 2, 1, 2, 1, 1, 2]]
[15, [1, 1, 0, 2, 1]]
[16, [2, 2, 2, 2, 1]]
[17, [2, 2, 1, 0, 0, 1, 1, 2, 2, 2]]
[18, [1, 2, 2, 0, 2, 2, 2]]
[19, [1, 1, 2, 2, 2, 0, 2, 2, 1]]
[20, [1, 0, 1, 2, 1, 2, 2, 2]]
[21, [2, 2, 1, 2, 1, 1, 2]]
[22, [2, 0, 2, 1, 0, 1, 0, 0, 0, 1]]
[23, [1, 1, 1, 0, 1, 1, 0, 1, 2, 0, 1, 1, 1]]
[24, [2, 0, 0, 1]]
[25, [1, 2, 0, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 2, 0, 2]]
[26, [2, 0, 0, 2, 1, 2, 1, 1, 0, 2]]
[27, [1, 1, 1, 0, 2, 2]]
[28, [2, 1, 1, 0, 0, 0, 0, 2, 2, 1]]
[29, [2, 2, 0, 0, 1, 1, 1, 0, 1, 0, 0, 2, 0, 1]]
[30, [1, 1, 0, 2, 1]]
[31, [1, 2, 0, 1, 0, 2, 0, 0, 1, 2]]


In [6]:
import pickle
rfile = open('/Users/barrybrent/22mar21no2.txt','r') # star polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..30]:
    n=s[k][0]
    poly=D(s[k][1])
    f = reduced(poly)
    nm=num(abs(substituteNumberForXInExpression(3,x,f)))
    vd3=nm.digits(3)
    vd3.reverse() 
    print([n,vd3])

[2, [2, 2]]
[3, [1, 0, 0, 1]]
[4, [2, 0, 0, 0, 1, 1, 2]]
[5, [2, 0, 1, 2, 1, 2, 2]]
[6, [2, 2, 0, 2, 2]]
[7, [2, 1, 1, 2, 2, 2, 0, 1, 1]]
[8, [1, 1, 0, 2, 1, 2, 1, 2, 2, 2]]
[9, [1, 2, 2, 0, 2, 2, 2]]
[10, [1, 2, 2, 2, 0, 0, 0, 0, 1]]
[11, [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1]]
[12, [2, 0, 0, 2, 1, 1, 2, 1, 1, 2]]
[13, [1, 0, 0, 2, 1, 0, 0, 1, 1, 1, 2, 0, 1]]
[14, [2, 0, 2, 1, 0, 2, 0, 2, 0, 1, 2]]
[15, [2, 0, 2, 1, 2, 1, 1, 1, 2, 2]]
[16, [1, 2, 1, 2, 0, 1, 1, 0, 0, 2, 1, 2, 1]]
[17, [1, 1, 0, 2, 2, 2, 2, 1, 2, 0, 1, 1, 1]]
[18, [1, 2, 0, 1, 0, 1, 2, 0, 1]]
[19, [2, 0, 2, 0, 0, 1, 1, 2, 2, 2, 0, 1, 1, 0, 2]]
[20, [1, 1, 1, 1, 0, 1, 0, 1, 2, 2, 0, 2, 1, 1]]
[21, [2, 1, 2, 2, 1, 1, 0, 1, 0, 0, 1, 1]]
[22, [2, 2, 0, 0, 1, 0, 2, 1, 2, 1, 0, 1, 2]]
[23, [1, 0, 2, 2, 0, 0, 2, 0, 1, 1, 2, 0, 2, 1, 2]]
[24, [1, 1, 1, 1, 0, 0, 1, 1, 2, 0, 2, 2, 2]]
[25, [1, 2, 0, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 2, 0, 2]]
[26, [2, 2, 2, 0, 0, 2, 1, 1, 0, 0, 1, 1, 2, 2]]
[27, [1, 2, 0, 0, 2, 2, 1, 2, 2, 2, 2]]
[28,

In [7]:
import pickle
rfile = open('/Users/barrybrent/22mar21no2.txt','r') # star polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..30]:
    n=s[k][0]
    poly=D(s[k][1])
    f = reduced(poly)
    dn=den(abs(substituteNumberForXInExpression(3,x,f)))
    vd3=dn.digits(3)
    vd3.reverse() 
    print([n,vd3])

[2, [1]]
[3, [1]]
[4, [1, 0, 0, 0]]
[5, [1, 0, 0, 0]]
[6, [1, 0, 0]]
[7, [1, 0, 0, 0, 0, 0, 0]]
[8, [1, 0, 0, 0, 0, 0, 0]]
[9, [1, 0, 0, 0, 0]]
[10, [1, 0, 0, 0, 0, 0, 0, 0]]
[11, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[12, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[13, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[14, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[15, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[16, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[17, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[18, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[19, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[20, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[21, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[22, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[23, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[24, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[25, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [8]:
import pickle
rfile = open('/Users/barrybrent/22mar21no2.txt','r') # star polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..30]:
    n=s[k][0]
    poly=D(s[k][1])
    f = reduced(poly)
    nm=num(abs(substituteNumberForXInExpression(n,x,f)))
    vd3=nm.digits(3)
    vd3.reverse() 
    print([n,vd3])

[2, []]
[3, [1, 0, 0, 1]]
[4, [1, 1, 0, 1, 2, 2, 1]]
[5, [2, 2, 2, 1, 0, 2, 1, 2]]
[6, [1, 0, 0, 1, 0, 0, 1, 1, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2]]
[7, [1, 0, 0, 1, 2, 0, 1, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 2, 1]]
[8, [1, 0, 2, 0, 2, 2, 0, 1, 2, 1, 0, 2, 2, 1, 1, 2, 0, 2, 1, 1, 0, 2, 1, 2, 0, 0, 1, 1]]
[9, [1, 0, 0, 2, 2, 1, 1, 1, 1, 0, 1, 1, 2, 0, 2, 2, 0, 2, 2, 1, 0, 2, 0, 1, 1, 2, 1, 0, 0, 1, 0, 2, 1, 2, 2, 0, 0, 0, 2]]
[10, [1, 1, 1, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 1, 1, 2, 0, 2, 2, 1, 0, 0, 2, 2, 2, 1, 0, 1, 2, 2, 2, 1, 0, 1, 1, 1, 2, 2, 1, 2, 0, 1]]
[11, [2, 0, 0, 0, 2, 1, 2, 1, 2, 2, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 2, 1, 1, 1, 1, 0, 2, 1, 0, 1, 0, 1, 2]]
[12, [2, 1, 0, 1, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0, 1, 1, 0, 2, 1, 2, 0, 1, 0, 0, 2, 1, 0, 2, 0, 2, 0, 2, 0, 1, 1, 0, 0, 2, 1, 0, 0, 1, 2, 2, 2, 1, 0, 2, 1, 1, 2, 0, 2, 2, 0, 0, 1, 1, 1, 1]]
[13, [1, 2, 0, 0, 0, 2, 2, 0, 0, 1, 1, 1, 1, 1, 2, 1, 0, 0, 0, 1, 0, 0, 2, 2, 1, 0, 0, 2, 0,